In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
from lower_case_dict import d_dict as D_dict
from lower_case_dict import D_ocupaciones
from lower_case_dict import d_describe as D_describe
from traduce import leyenda as tr_leyenda
pd.options.display.max_columns=None
pd.options.display.max_colwidth=310
pd.options.mode.chained_assignment=None

In [2]:
# Dataset completo procesado
ruta='data/formacion_procesado.csv'
df=pd.read_csv(ruta,index_col=0,low_memory=False)

# Naive decision tree

**Clasificador de ocupaciones a partir de títulos al estilo de un arbol de decisión**, con la diferencia de que, en lugar de identificar el punto de corte más explicativo en cada etapa, este clasificador solo reparte en ramas una vez y crea **tantas ramas como títulos distintos** haya en el dataset.   

Para cada título identifica todas las ocupaciones, las ordena de más a menos frecuentes y  selecciona como predicción las ocupaciones en el **top n del ranking de frecuencias** que, además, estén por encima de un **umbral de frecuencia relativa establecido como mínimo**.   

Para cada título, este clasificador devuelve como **predicción una lista de ocupaciones ordenadas** de más a menos probables.   

Los **parámetros de este modelo** son el número n de posiciones en el ranking que se seleccionan como predictores y el umbral de frecuencia mínima.   

El número de ocupaciones en la predicción para cada título puede ser mayor o menor que  n, el número de posiciones en el ranking de frecuencias que se tienen en cuenta:   

- Si alguna ocupación en el top n del ranking tiene una frecuencia inferior al umbral, se descarta como predictor. Así, es posible que la lista de ocupaciones seleccionadas tenga menos de n elementos.  

- Cuando hay empates entre ocupaciones en el top n de frecuencias, se incluyen en la predicción todas las que pertenecen al grupo empatado. Esto hace posible que se seleccionen más de n ocupaciones como predicción. 

El algoritmo se ha construido sobre las columnas de título y de ocupacion literales en lugar de las nuḿericas. Se ha decidido así porque facilita el hacerse una idea de cuáles son las titulaciones para las que es más o menos probable hacer predicciones acertadas durante el proceso de construcción del clasificador. 

## Objetivo

El objetivo de la construcción de un clasificador elemental es conocer en qué medida las titulaciones universitarias explican por sí solas las profesiones que alcanzan los titulados.   
Se espera un éxito modesto por varios motivos:   

- Los títulos universitarios no son el único critero de los empleadores para seleccionar a su plantilla. Por un lado, un título universitario suele ser insuficiente por sí solo para aspirar a un buen número de puestos de trabajo reservados a universitarios. Por otro lado, a menudo los candidatos a un empleo pueden suplir la falta de un título universitario especializado a menudo con otros méritos, como la experiencia profesional o la formación en disciplinas afines. Los estudios de medicina son una excepción clara a esta posibilidad; los resultados constatan que las profesiones sanitarias están reservadas a titulados con formación especializada (aunque no todos los que tienen esa formación trabajan en ocupaciones relacionadas con su título). 

- Las encuestas empleadas en este proyecto solo informan con detalle de un título universitario de cada encuestado. Aquellos que han accedido a su profesión gracias a un título distinto del que les hace elegibles para la encuesta en la que participan (otro grado, otro máster, un título de doctor) se escapan a la capacidad predictiva de los modelos basados en estos datos.  Esto se notará especialmente en los casos en los que el encuestado responde sobre un título que ha estudiado por motivos distintos a búsqueda de empleo o la promoción profesional mientras trabajaban en ocupaciones a las que accedieron gracias a una titulación previa.  

- El mal funcionamiento del mercado de trabajo español empuja a muchos aspirantes a un empleo a aceptar trabajos poco o nada relacionados con su formación y sus conocimientos.   


## Justificación
**Ventajas de la construcción del algoritmo a medida** (en lugar de emplear una solución estandarizada):  
- Simplicidad de la justificación teórica que sustenta las predicciones. Facilita la interpretación de los resultados. 
- Flexibilidad para definir los criterios de agregación de las métricas de evaluación, por titulaciones o por ocupaciones. Facilita la identificación de los títulos y las profesiones para las que es más y menos probable hacer predicciones acertadas. 

## Dataset

#### Observaciones con target informado

In [3]:
# Selección: solo ocupaciones informadas
df=df[df['ocupacion'].notna()]
df.shape

(59844, 352)

#### Número de categorías distintas del target: 

In [4]:
df['ocupacion'].nunique()
# Ocupaciones distintas (núm. categorías del target):

62

#### Número de categorías distintas del regresor: 

In [5]:
df['titulo_ppal_'].nunique()
# Titulos distintos (num. categorías del regresor principal): 

173

## Entrenamiento, validación y test: 
Se separan los dataset de train, validación y test sin distinguir regresores y target porque es más conveniente mantener todas las columnas en un dataframe único para aplicar el algoritmo y evaluar los resultados. 

In [6]:
df=df.sample(frac=1,random_state=23)
n_train=int(df.shape[0]*0.6)
n_vld=int(df.shape[0]*0.8)
df_train=df[:n_train]
df_vld=df[n_train:n_vld]
df_test=df[n_vld:]
df_train.shape[0]+df_vld.shape[0]+df_test.shape[0]==df.shape[0] # Comprobación

True

In [7]:
df_train.shape[0], df_vld.shape[0], df_test.shape[0]

(35906, 11969, 11969)

## Número de ocupaciones distintas por título: 

In [8]:
columnas=['titulo_ppal_','ocupacion_']
df_num_oc_por_titulo=df_train[columnas].groupby('titulo_ppal_').agg(['nunique'])
df_num_oc_por_titulo.columns=df_num_oc_por_titulo.columns.to_flat_index() 
df_num_oc_por_titulo.rename(columns={df_num_oc_por_titulo.columns[-1]:'ocupaciones por titulo'},inplace=True)
df_num_oc_por_titulo.sort_values(by='ocupaciones por titulo',inplace=True)
df_num_oc_por_titulo.reset_index(inplace=True)
df_num_oc_por_titulo

,titulo_ppal_,ocupaciones por titulo
0,Máster Odontología,3
1,Grado Odontología,4
2,"Máster Ciencias químicas, físicas y geológicas; Matemáticas y estadística (Otros estudios)",5
3,Máster Enfermería,5
4,Máster Religión y teología,6
...,...,...
168,Grado Arquitectura técnica,47
169,Grado Educación primaria,47
170,Grado Historia,48
171,Grado Economía,51


## Número de ocupaciones distintas por título: resumen 

In [9]:
df_num_oc_por_titulo['ocupaciones por titulo'].describe()
# núm. de ocupaciones únicas agrupadas por título ppal (resumen):
# count= número de cardinales (distintos) de ocupaciones agrupadas por título principal

count    173.000000
mean      23.057803
std       11.800035
min        3.000000
25%       14.000000
50%       21.000000
75%       30.000000
max       51.000000
Name: ocupaciones por titulo, dtype: float64

## Número de titulados por título: 

In [10]:
columnas=['titulo_ppal_','ocupacion_']
df_titulados=df_train[columnas].groupby(['titulo_ppal_']).agg(['count'])
df_titulados.columns=df_titulados.columns.to_flat_index()
df_titulados.reset_index(inplace=True)
df_titulados.rename(columns={df_titulados.columns[-1]:'titulados'},inplace=True)
df_titulados

,titulo_ppal_,titulados
0,Grado Actividad física y del deporte,573
1,Grado Administración y empresa,1571
2,Grado Antropología social y cultural y Estudios y gestión de la cultura,158
3,Grado Arqueología,20
4,Grado Arquitectura técnica,623
...,...,...
168,Máster Técnicas audiovisuales y medios de comunicación,57
169,"Máster Vehículos de motor, barcos y aeronaves",39
170,Máster Ventas al por mayor y al por menor,33
171,Máster Veterinaria,32


In [11]:
df_titulados=df_titulados.merge(df_num_oc_por_titulo,on='titulo_ppal_',how='outer')

In [12]:
df_titulados=df_titulados.sort_values(by='ocupaciones por titulo')
df_titulados

,titulo_ppal_,titulados,ocupaciones por titulo
151,Máster Odontología,39,3
80,Grado Odontología,144,4
114,"Máster Ciencias químicas, físicas y geológicas; Matemáticas y estadística (Otros estudios)",13,5
126,Máster Enfermería,89,5
158,Máster Religión y teología,17,6
...,...,...,...
4,Grado Arquitectura técnica,623,47
25,Grado Educación primaria,1241,47
40,Grado Historia,411,48
23,Grado Economía,722,51


## Número de profesionales por título y ocupación: 

In [13]:
columnas=['titulo_ppal_','ocupacion_']
df_oc_por_titulo=df_train[columnas].groupby(['titulo_ppal_']).agg(['value_counts'])
df_oc_por_titulo.columns=df_oc_por_titulo.columns.to_flat_index()
df_oc_por_titulo.reset_index(inplace=True)
df_oc_por_titulo.rename(columns={df_oc_por_titulo.columns[-1]:'profesionales'},inplace=True)
df_oc_por_titulo

,titulo_ppal_,ocupacion_,profesionales
0,Grado Actividad física y del deporte,"Prof apoyo servicios jurídicos, socio-culturales, deportivos",218
1,Grado Actividad física y del deporte,Prof enseñanza,135
2,Grado Actividad física y del deporte,Trabajadores protección y seguridad,27
3,Grado Actividad física y del deporte,Dependientes de comercio,24
4,Grado Actividad física y del deporte,Prof sanitarios,22
...,...,...,...
3984,"Máster Viajes, turismo y ocio",Trabajadores protección y seguridad,1
3985,"Máster Viajes, turismo y ocio",Trabajadores servicios personales,1
3986,"Máster Viajes, turismo y ocio",Técnicos TI,1
3987,"Máster Viajes, turismo y ocio",Técnicos ciencias e ingenierías,1


In [14]:
df_oc_por_titulo=df_titulados.merge(df_oc_por_titulo,on='titulo_ppal_',how='outer')

In [15]:
df_oc_por_titulo.sort_values(by=['ocupaciones por titulo','profesionales'],ascending=[True,False],inplace=True)
df_oc_por_titulo

,titulo_ppal_,titulados,ocupaciones por titulo,ocupacion_,profesionales
0,Máster Odontología,39,3,Prof sanitarios,37
1,Máster Odontología,39,3,Política y dirección Admón Pública,1
2,Máster Odontología,39,3,Prof enseñanza,1
3,Grado Odontología,144,4,Prof sanitarios,140
4,Grado Odontología,144,4,Prof enseñanza,2
...,...,...,...,...,...
3984,Grado Administración y empresa,1571,51,Peones de industrias manufactureras,1
3985,Grado Administración y empresa,1571,51,Trabajadores construcción e instalaciones: acabado,1
3986,Grado Administración y empresa,1571,51,Trabajadores construcción y afines: estructura,1
3987,Grado Administración y empresa,1571,51,Trabajadores cualificados ganadería,1


## Proporción de profesionales sobre titulados por ocupación y título: 

In [16]:
df_oc_por_titulo['prop_oc_titulados']=df_oc_por_titulo['profesionales'].div(df_oc_por_titulo['titulados'])
df_oc_por_titulo

,titulo_ppal_,titulados,ocupaciones por titulo,ocupacion_,profesionales,prop_oc_titulados
0,Máster Odontología,39,3,Prof sanitarios,37,0.948718
1,Máster Odontología,39,3,Política y dirección Admón Pública,1,0.025641
2,Máster Odontología,39,3,Prof enseñanza,1,0.025641
3,Grado Odontología,144,4,Prof sanitarios,140,0.972222
4,Grado Odontología,144,4,Prof enseñanza,2,0.013889
...,...,...,...,...,...,...
3984,Grado Administración y empresa,1571,51,Peones de industrias manufactureras,1,0.000637
3985,Grado Administración y empresa,1571,51,Trabajadores construcción e instalaciones: acabado,1,0.000637
3986,Grado Administración y empresa,1571,51,Trabajadores construcción y afines: estructura,1,0.000637
3987,Grado Administración y empresa,1571,51,Trabajadores cualificados ganadería,1,0.000637


### Proporciones acumuladas: 

In [17]:
df_prop_acum=df_oc_por_titulo[['titulo_ppal_','prop_oc_titulados']].groupby('titulo_ppal_').cumsum()
df_prop_acum.rename(columns={df_prop_acum.columns[-1]:'prop_acum'},inplace=True)
df_prop_acum

,prop_acum
0,0.948718
1,0.974359
2,1.000000
3,0.972222
4,0.986111
...,...
3984,0.997454
3985,0.998090
3986,0.998727
3987,0.999363


In [18]:
df_oc_por_titulo=df_oc_por_titulo.merge(df_prop_acum,left_index=True,right_index=True,how='outer')
df_oc_por_titulo

,titulo_ppal_,titulados,ocupaciones por titulo,ocupacion_,profesionales,prop_oc_titulados,prop_acum
0,Máster Odontología,39,3,Prof sanitarios,37,0.948718,0.948718
1,Máster Odontología,39,3,Política y dirección Admón Pública,1,0.025641,0.974359
2,Máster Odontología,39,3,Prof enseñanza,1,0.025641,1.000000
3,Grado Odontología,144,4,Prof sanitarios,140,0.972222,0.972222
4,Grado Odontología,144,4,Prof enseñanza,2,0.013889,0.986111
...,...,...,...,...,...,...,...
3984,Grado Administración y empresa,1571,51,Peones de industrias manufactureras,1,0.000637,0.997454
3985,Grado Administración y empresa,1571,51,Trabajadores construcción e instalaciones: acabado,1,0.000637,0.998090
3986,Grado Administración y empresa,1571,51,Trabajadores construcción y afines: estructura,1,0.000637,0.998727
3987,Grado Administración y empresa,1571,51,Trabajadores cualificados ganadería,1,0.000637,0.999363


## Ranking de ocupaciones por título: 

In [19]:
df_rank=df_oc_por_titulo[['titulo_ppal_','profesionales']]
df_rank=df_rank.groupby('titulo_ppal_').rank(method='min',ascending=False)
df_rank.rename(columns={df_rank.columns[-1]:'rank'},inplace=True)
df_rank

,rank
0,1.0
1,2.0
2,2.0
3,1.0
4,2.0
...,...
3984,44.0
3985,44.0
3986,44.0
3987,44.0


In [20]:
df_oc_por_titulo=df_oc_por_titulo.merge(df_rank,how='outer',left_index=True,right_index=True)

In [21]:
df_oc_por_titulo.sort_values(by=['ocupaciones por titulo','rank'],inplace=True)
df_oc_por_titulo

,titulo_ppal_,titulados,ocupaciones por titulo,ocupacion_,profesionales,prop_oc_titulados,prop_acum,rank
0,Máster Odontología,39,3,Prof sanitarios,37,0.948718,0.948718,1.0
1,Máster Odontología,39,3,Política y dirección Admón Pública,1,0.025641,0.974359,2.0
2,Máster Odontología,39,3,Prof enseñanza,1,0.025641,1.000000,2.0
3,Grado Odontología,144,4,Prof sanitarios,140,0.972222,0.972222,1.0
4,Grado Odontología,144,4,Prof enseñanza,2,0.013889,0.986111,2.0
...,...,...,...,...,...,...,...,...
3984,Grado Administración y empresa,1571,51,Peones de industrias manufactureras,1,0.000637,0.997454,44.0
3985,Grado Administración y empresa,1571,51,Trabajadores construcción e instalaciones: acabado,1,0.000637,0.998090,44.0
3986,Grado Administración y empresa,1571,51,Trabajadores construcción y afines: estructura,1,0.000637,0.998727,44.0
3987,Grado Administración y empresa,1571,51,Trabajadores cualificados ganadería,1,0.000637,0.999363,44.0


# Ranking de ocupaciones más frecuentes por título (clasificador naive) 

In [23]:
max_rank=5
min_prop=0.05
naive_top_n=df_oc_por_titulo[df_oc_por_titulo['rank']<=max_rank]
naive_top_n=naive_top_n[naive_top_n['prop_oc_titulados']>=min_prop]
naive_top_n.sort_values(by=['ocupaciones por titulo','titulo_ppal_','rank'],inplace=True)
naive_top_n.reset_index(drop=True,inplace=True)
naive_top_n.sample(2)

,titulo_ppal_,titulados,ocupaciones por titulo,ocupacion_,profesionales,prop_oc_titulados,prop_acum,rank
699,Grado Economía,722,51,Prof ciencias sociales,66,0.091413,0.463989,3.0
70,Grado Arqueología,20,10,Dependientes de comercio,1,0.050000,0.800000,4.0


In [ ]:
#with pd.option_context("display.max_rows", naive_top_n.shape[0]):
#    display(naive_top_n)

# Predicciones 

In [24]:
def prediccion_top_n(titulo):
    prd=naive_top_n[naive_top_n['titulo_ppal_']==titulo]
    prd=prd[['ocupacion_','prop_oc_titulados']]
    prd.reset_index(drop=True,inplace=True)
    prd.rename(columns={'ocupacion_':'prediccion','prop_oc_titulados':'probabilidad'},inplace=True)
    return prd

In [25]:
# Inspección: 
prediccion_top_n('Máster Odontología')

,prediccion,probabilidad
0,Prof sanitarios,0.948718


In [26]:
# Inspección: 
prediccion_top_n('Grado Economía')

,prediccion,probabilidad
0,"Empleados servicios financieros, contables y de apoyo",0.270083
1,"Expertos en gestión, finanzas, comercio, etc",0.102493
2,Prof ciencias sociales,0.091413
3,Empleados administrativos sin atención al público,0.070637
4,Otros empleados administrativos con atención al público,0.069252


In [27]:
# Inspección: 
prediccion_top_n('Grado Matemáticas')

,prediccion,probabilidad
0,Prof enseñanza,0.350820
1,"Prof ciencias, matemáticas e ingeniería",0.180328
2,Técnicos TI,0.101639
3,Profesionales IT,0.081967
4,Prof eduación especial y no reglada,0.059016


# Evaluación del clasificador

In [28]:
def evalua_top_n(data):
    
    evalua=data[['titulo_ppal_','ocupacion_']]
    evalua['prediccion']=evalua['titulo_ppal_'].apply(lambda x: prediccion_top_n(x)['prediccion'].tolist())
    
    evalua['acierto']=evalua.apply(lambda x: x['ocupacion_'] in x['prediccion'],axis=1).astype(int)
    
    aux_aciertos_titulo=evalua[['titulo_ppal_','acierto']].groupby('titulo_ppal_').agg(['sum'])
    aux_aciertos_titulo.columns=aux_aciertos_titulo.columns.to_flat_index()
    evalua=evalua.merge(aux_aciertos_titulo,on='titulo_ppal_',how='outer')
    evalua.rename(columns={evalua.columns[-1]:'aciertos titulo'},inplace=True)
    
    aux_total_titulados=evalua[['titulo_ppal_','acierto']].groupby('titulo_ppal_').agg(['count'])
    aux_total_titulados.columns=aux_total_titulados.columns.to_flat_index()
    evalua=evalua.merge(aux_total_titulados,on='titulo_ppal_',how='outer')
    evalua.rename(columns={evalua.columns[-1]:'num titulados'},inplace=True)
    
    evalua['accuracy titulo']=evalua['aciertos titulo'].div(evalua['num titulados'])
    
    total_obs=evalua.shape[0]
    
    evalua['peso titulo']=evalua['num titulados']/total_obs
    evalua['accuracy ponderado titulo']=evalua['accuracy titulo']*evalua['peso titulo']
    
    aux_aciertos_ocupacion=evalua[['ocupacion_','acierto']].groupby('ocupacion_').agg(['sum'])
    aux_aciertos_ocupacion.columns=aux_aciertos_ocupacion.columns.to_flat_index()
    evalua=evalua.merge(aux_aciertos_ocupacion,on='ocupacion_',how='outer')
    evalua.rename(columns={evalua.columns[-1]:'aciertos ocupacion'},inplace=True)
    
    aux_total_profesionales=evalua[['ocupacion_','acierto']].groupby('ocupacion_').agg(['count'])
    aux_total_profesionales.columns=aux_total_profesionales.columns.to_flat_index()
    evalua=evalua.merge(aux_total_profesionales,on='ocupacion_',how='outer')
    evalua.rename(columns={evalua.columns[-1]:'num profesionales'},inplace=True)
  
    evalua['accuracy ocupacion']=evalua['aciertos ocupacion'].div(evalua['num profesionales'])

    evalua['peso ocupacion']=evalua['num profesionales']/total_obs
    evalua['accuracy ponderado ocupacion']=evalua['accuracy ocupacion']*evalua['peso ocupacion']
    

    return evalua

### Evaluación desagregada (en train):

In [29]:
df_train_evalua_top_n=evalua_top_n(df_train)
df_train_evalua_top_n.sample(2)

,titulo_ppal_,ocupacion_,prediccion,acierto,aciertos titulo,num titulados,accuracy titulo,peso titulo,accuracy ponderado titulo,aciertos ocupacion,num profesionales,accuracy ocupacion,peso ocupacion,accuracy ponderado ocupacion
20231,Grado Ingeniería electrónica industrial y automática,Prof apoyo gestión administrativa,"[Prof ciencias, matemáticas e ingeniería, Técnicos ciencias e ingenierías]",0,500,716,0.698324,0.019941,0.013925,21,370,0.056757,0.010305,0.000585
35419,Grado Información y documentación,Prof cultura y espectáculo,"[Prof cultura y espectáculo, Empleados bibliotecas, correos, etc, Empleados administrativos sin atención al público, Expertos en gestión, finanzas, comercio, etc]",1,57,108,0.527778,0.003008,0.001587,499,689,0.724238,0.019189,0.013897


In [30]:
df_train_evalua_top_n['acierto'].sum()/df_train_evalua_top_n['acierto'].count()

0.6767671141313429

### Evaluación desagregada (en validación):

In [31]:
df_vld_evalua_top_n=evalua_top_n(df_vld)
df_vld_evalua_top_n.sample(2)

,titulo_ppal_,ocupacion_,prediccion,acierto,aciertos titulo,num titulados,accuracy titulo,peso titulo,accuracy ponderado titulo,aciertos ocupacion,num profesionales,accuracy ocupacion,peso ocupacion,accuracy ponderado ocupacion
8668,Máster Tecnologías de la información y las comunicaciones (Otros estudios),Profesionales IT,"[Profesionales IT, Prof ciencias, matemáticas e ingeniería, Técnicos TI, Directores producción y operaciones]",1,22,29,0.758621,0.002423,0.001838,303,374,0.810160,0.031247,0.025315
480,Grado Arquitectura técnica,"Prof ciencias, matemáticas e ingeniería","[Prof ciencias, matemáticas e ingeniería, Técnicos ciencias e ingenierías]",1,123,208,0.591346,0.017378,0.010277,1419,1526,0.929882,0.127496,0.118556


In [32]:
df_vld_evalua_top_n['acierto'].sum()/df_vld_evalua_top_n['acierto'].count()

0.6675578577993149

### Métricas de evaluación por títulos y por ocupaciones: 

In [33]:
def evalua_top_n_regresor(data):
    evalua=evalua_top_n(data)
    lista_re_regresor=['titulo_ppal_','aciertos titulo','num titulados','accuracy titulo','peso titulo',\
                       'accuracy ponderado titulo']
    evalua_regresor=evalua[lista_re_regresor].drop_duplicates()
    evalua_regresor.set_index('titulo_ppal_',inplace=True)

    return evalua_regresor

In [34]:
df_vld_evalua_top_n_regresor=evalua_top_n_regresor(df_vld)
df_vld_evalua_top_n_regresor

,aciertos titulo,num titulados,accuracy titulo,peso titulo,accuracy ponderado titulo
titulo_ppal_,,,,,
Grado Ingeniería forestal y montes,56,85,0.658824,0.007102,0.004679
Grado Ingeniería aeronáutica,56,74,0.756757,0.006183,0.004679
Grado Derecho,307,430,0.713953,0.035926,0.025650
Grado Química,88,136,0.647059,0.011363,0.007352
Máster Dirección y administración,67,112,0.598214,0.009358,0.005598
...,...,...,...,...,...
Grado Desarrollo de software y de aplicaciones e Ingeniería multimedia,9,10,0.900000,0.000835,0.000752
Máster Religión y teología,3,4,0.750000,0.000334,0.000251
Grado Lenguas clásicas,18,26,0.692308,0.002172,0.001504


In [35]:
def evalua_top_n_target(data):
    evalua=evalua_top_n(data)
    lista_re_target=['ocupacion_','aciertos ocupacion','num profesionales','accuracy ocupacion','peso ocupacion',
                     'accuracy ponderado ocupacion']
    evalua_target=evalua[lista_re_target].drop_duplicates()
    evalua_target.set_index('ocupacion_',inplace=True)

    return evalua_target

In [36]:
df_vld_evalua_top_n_target=evalua_top_n_target(df_vld)
df_vld_evalua_top_n_target

,aciertos ocupacion,num profesionales,accuracy ocupacion,peso ocupacion,accuracy ponderado ocupacion
ocupacion_,,,,,
"Prof ciencias, matemáticas e ingeniería",1419,1526,0.929882,0.127496,0.118556
Técnicos ciencias e ingenierías,316,401,0.788030,0.033503,0.026402
Trabajadores servicios de cuidados de salud,0,65,0.000000,0.005431,0.000000
Vendedores excepto comercio,0,34,0.000000,0.002841,0.000000
Gerentes hostelería y comercio,0,35,0.000000,0.002924,0.000000
...,...,...,...,...,...
Trabajadores cualificados ganadería,0,8,0.000000,0.000668,0.000000
Trabajadores construcción e instalaciones: acabado,0,6,0.000000,0.000501,0.000000
"Trabajadores madera, textil, confección, piel y cuero",0,6,0.000000,0.000501,0.000000


## Resúmenes de métricas de evaluación por titulaciones y por ocupaciones

In [37]:
def evalua_top_n_regresor_resumen(data):
    evalua_regresor=evalua_top_n_regresor(data)
    evalua_resumen=evalua_regresor.agg(['sum','mean','median','min','argmin','max','argmax'])
    # Ajustes argmin y argmax: 
    for col in evalua_resumen.columns: 
        for arg in ['argmin','argmax']:
            indice_titulo=int(evalua_resumen.loc[arg,col])
            evalua_resumen.loc[arg,col]=evalua_regresor.index[indice_titulo]
    # Ajustes 'no aplica' (sumas o índices que no tienen sentido):
    evalua_resumen.loc['sum','accuracy titulo']=''

    return evalua_resumen

In [38]:
resumen_regresor_validacion= evalua_top_n_regresor_resumen(df_vld)
resumen_regresor_validacion

,aciertos titulo,num titulados,accuracy titulo,peso titulo,accuracy ponderado titulo
sum,7990,11969,,1,0.667558
mean,46.185,69.185,0.648427,0.00578035,0.00385872
median,19,30,0.666667,0.00250648,0.00158743
min,0,4,0,0.000334197,0
argmin,Máster Artes (Otros estudios),Máster Religión y teología,Máster Artes (Otros estudios),Máster Religión y teología,Máster Artes (Otros estudios)
max,398,526,0.983974,0.0439469,0.0332526
argmax,Grado Enfermería,Grado Administración y empresa,Grado Medicina,Grado Administración y empresa,Grado Enfermería


In [39]:
def evalua_top_n_target_resumen(data):
    evalua_target=evalua_top_n_target(data)
    evalua_resumen=evalua_target.agg(['sum','mean','median','min','argmin','max','argmax'])
    # Ajustes argmin y argmax: 
    for col in evalua_resumen.columns: 
        for arg in ['argmin','argmax']:
            indice_ocupacion=int(evalua_resumen.loc[arg,col])
            evalua_resumen.loc[arg,col]=evalua_target.index[indice_ocupacion]
    # Ajustes 'no aplica' (sumas o índices que no tienen sentido):
    evalua_resumen.loc['sum','accuracy ocupacion']=''

    return evalua_resumen

In [40]:
resumen_target_validacion=evalua_top_n_target_resumen(df_vld)
resumen_target_validacion

,aciertos ocupacion,num profesionales,accuracy ocupacion,peso ocupacion,accuracy ponderado ocupacion
sum,7990,11969,,1,0.667558
mean,128.871,193.048,0.206963,0.016129,0.0107671
median,0,43.5,0,0.00363439,0
min,0,3,0,0.000250648,0
argmin,Trabajadores servicios de cuidados de salud,Montadores y ensambladores en fábricas,Trabajadores servicios de cuidados de salud,Montadores y ensambladores en fábricas,Trabajadores servicios de cuidados de salud
max,1775,1963,0.944481,0.164007,0.1483
argmax,Prof enseñanza,Prof enseñanza,Prof sanitarios,Prof enseñanza,Prof enseñanza
